In [1]:
import os

# 获取当前工作目录
current_dir = os.getcwd()
print("当前工作目录：", current_dir)

# 切换到上一层目录
parent_dir = os.path.dirname(current_dir)
os.chdir(parent_dir)
print("切换后的目录：", parent_dir)

当前工作目录： /Users/lig/Documents/GitHub/NEMESIS/unit_test
切换后的目录： /Users/lig/Documents/GitHub/NEMESIS


## Development

In [2]:
import QuantLib as ql
import pandas as pd
import numpy as np
from devlib.market.curves.pm_curves import PmForwardCurve
from devlib.products.fx import fx_vanilla, fx_digital, fx_range_accrual, fx_knock
from devlib.market.volatility.fx_vol_surface import FxVolSurfaceHK
from devlib.market.volatility.constant_vol_surface import ConstantVolSurface
from devlib.market.curves.fx_curves import FxImpliedAssetCurve
from devlib.market.curves.overnight_index_curves import Sofr
from devlib.utils.fx_utils import get_calendar

In [3]:
def dev_valuation(inst, today, forward_curve, d_curve, vol_surface, daycount=ql.Actual365Fixed()):
    npv = inst.npv(today, forward_curve, d_curve, vol_surface, daycount)
    delta = inst.delta(today, forward_curve, d_curve, vol_surface, daycount, tweak=1)
    gamma = inst.gamma_1pct(today, forward_curve, d_curve, vol_surface, daycount, tweak=1)
    vega = inst.vega(today, forward_curve, d_curve, vol_surface, daycount, tweak=1)
    theta = inst.theta(today, forward_curve, d_curve, vol_surface, daycount, tweak=1)
    rho1 = inst.rho(today, forward_curve, d_curve, vol_surface, daycount, tweak=100, tweak_type='market') * 100
    rho2 = inst.rho(today, forward_curve, d_curve, vol_surface, daycount, tweak=100, tweak_type='pillar_rate') * 100
    phi1 = inst.phi(today, forward_curve, d_curve, vol_surface, daycount, tweak=100, tweak_type='market') * 100
    phi2 = inst.phi(today, forward_curve, d_curve, vol_surface, daycount, tweak=100, tweak_type='pillar_rate') * 100

    print(f'npv: {npv}')
    print(f'delta: {delta}')
    print(f'gamma: {gamma}')
    print(f'vega: {vega}')
    print(f'theta: {theta}')
    print(f'rho(market): {rho1}')
    print(f'rho(pillar_rate): {rho2}')
    print(f'phi(market): {phi1}')
    print(f'phi(pillar_rate): {phi2}')

In [4]:
today = ql.Date(11,8,2025)
ql.Settings.instance().evaluationDate = today

# pm forward_curve
d_ccy = 'USD'
f_ccy = 'XAU'
mkt_file_path = './unit_test/data/market_data_fivs_20250811.xlsx'
pm_fwd_data = pd.read_excel(mkt_file_path, sheet_name='precious metal')
pm_fwd_data = pm_fwd_data.loc[pm_fwd_data['TYPE'] == f_ccy+d_ccy, ['TENOR', 'SETTLE_DT', 'PX_LAST']]
pm_fwd_data.columns = ['Tenor', 'SettleDate', 'Rate']
pm_fwd_data.loc[pm_fwd_data.loc[:,'Tenor']!='SPOT', 'Rate'] /= 100
spot = pm_fwd_data.loc[pm_fwd_data.loc[:,'Tenor']=='SPOT', 'Rate'].values[0]
calendar = get_calendar(f_ccy, d_ccy, is_with_usd=True)
fx_fwd_crv = PmForwardCurve(today, spot, pm_fwd_data, f_ccy, d_ccy, calendar=calendar, daycount=ql.Actual365Fixed(), data_type='rate')

# d_curve
mkt_file = "./unit_test/data/sofr_curve_data_20250811.xlsx"
swap_mkt_data = pd.read_excel(mkt_file, sheet_name="swap")
fixing_data = pd.read_excel(mkt_file, sheet_name="fixing")
d_crv = Sofr(today, swap_mkt_data=swap_mkt_data, fixing_data=fixing_data)

# f_curve
calendar = get_calendar(f_ccy, d_ccy, is_with_usd=True)
f_crv = FxImpliedAssetCurve(today, d_crv, fx_fwd_crv, calendar, ql.Actual365Fixed())

# vol_surface
vol_file = "./unit_test/data/XAUUSD_voldata_20250811.xlsx"
vol_data = pd.read_excel(vol_file, sheet_name="vol_data")
calendar = get_calendar(f_ccy, d_ccy, is_with_usd=True)
vol_surface = FxVolSurfaceHK(today, vol_data, spot, d_ccy, f_ccy,
                             fx_fwd_crv, d_crv, f_crv, calendar, daycount=ql.Actual365Fixed())

In [5]:
expiry = ql.Date(13,11,2025)
payment_date = calendar.advance(expiry, ql.Period(2, ql.Days))
flavor = 'call'
strike = 3200
notional = 1e4
notional_ccy = 'USD'
trade_direction = 'long'

inst = fx_vanilla.FxVanilla(d_ccy, f_ccy, calendar, expiry, payment_date, flavor, strike,
                            notional, notional_ccy, trade_direction)
dev_valuation(inst, today, fx_fwd_crv, d_crv, vol_surface, daycount=ql.Actual365Fixed())

npv: 664.8243657925801
delta: 2.435173720414241
gamma: 0.136034084334824
vega: 15.719033583592307
theta: -1.1660791819493852
rho(market): 20.11992239335791
rho(pillar_rate): 20.06686594283434
phi(market): -21.910180507725272
phi(pillar_rate): -21.8522735786529


In [6]:
expiry = ql.Date(13,11,2025)
payment_date = calendar.advance(expiry, ql.Period(2, ql.Days))
flavor = 'call'
strike = 3200
pay_equal = False
cash = 3e4
cash_ccy = 'USD'
trade_direction = 'long'
cash_settle = True

inst = fx_digital.FxBinary(d_ccy, f_ccy, calendar, expiry, payment_date, flavor, strike,
                            pay_equal, cash, cash_ccy, trade_direction, cash_settle)
dev_valuation(inst, today, fx_fwd_crv, d_crv, vol_surface, daycount=ql.Actual365Fixed())

npv: 22366.58556009532
delta: 37.033463740954176
gamma: -3.7451171374414116
vega: -480.18813149324524
theta: 40.85719475346923
rho(market): 259.57650909244694
rho(pillar_rate): 258.7004465673126
phi(market): -319.7840971964433
phi(pillar_rate): -318.74208059385273


In [7]:
expiry = ql.Date(13,11,2025)
payment_date = calendar.advance(expiry, ql.Period(2, ql.Days))
strike = 3200
left_in = False
coupon_left = 3e4
coupon_right = 1e4
cash_ccy = 'USD'
trade_direction = 'long'
cash_settle = True

inst = fx_digital.FxDigital(d_ccy, f_ccy, calendar, expiry, payment_date, strike,
                            left_in, coupon_left, coupon_right, cash_ccy, trade_direction, cash_settle)
dev_valuation(inst, today, fx_fwd_crv, d_crv, vol_surface, daycount=ql.Actual365Fixed())

npv: 14747.049069556706
delta: -24.688975845492678
gamma: 2.4926916012191214
vega: 320.1254209954968
theta: -23.66051510177749
rho(market): -252.90416839532463
rho(pillar_rate): -252.097043578975
phi(market): 213.18939813096404
phi(pillar_rate): 212.4947203959027


In [8]:
expiry = ql.Date(13,11,2025)
payment_date = calendar.advance(expiry, ql.Period(2, ql.Days))
barrier_type = 'upout'
barrier = 3100
barrier_at_coupon = True
flavor = 'put'
strike = 3200
notional = 1e6
notional_ccy = 'USD'
trade_direction = 'long'

inst = fx_knock.FxKnock(d_ccy, f_ccy, calendar, expiry, payment_date,
                        barrier_type, barrier, barrier_at_coupon, flavor, strike,
                        notional, notional_ccy, trade_direction)
vol_surface = ConstantVolSurface(today, 0.15)
dev_valuation(inst, today, fx_fwd_crv, d_crv, vol_surface, daycount=ql.Actual365Fixed())

npv: 9062.90991656707
delta: -64.04820057468896
gamma: 11.405584009480663
vega: 1586.706942198874
theta: -125.82241891735976
rho(market): -577.687224934597
rho(pillar_rate): -575.7950191176951
phi(market): 553.2449160930219
phi(pillar_rate): 551.4249932358298


In [9]:
start_date = ql.Date(10,11,2025)
end_date = ql.Date(13,11,2025)
obs_end_date = end_date
obs_freq = ql.Period('1D')
obs_schedule = np.array(ql.Schedule(start_date, obs_end_date, obs_freq,
                                    calendar, ql.ModifiedFollowing, ql.ModifiedFollowing,
                                    ql.DateGeneration.Forward, False).dates())
payment_date = calendar.advance(end_date, ql.Period(2, ql.Days))

trade_direction = 'long'
notional = 1e6
cash_ccy = 'USD'
cash_settle = True

range_down = 3250
down_in = True
range_up = 3350
up_in = True
range_in_coupon_rate = 1
range_out_coupon_rate = 0
range_in_coupon = range_in_coupon_rate * notional
range_out_coupon = range_out_coupon_rate * notional

fx_fixing = pd.Series([spot], index=[today])

inst = fx_range_accrual.FxRangeAccrual(d_ccy, f_ccy, calendar, obs_schedule, payment_date,
                                       range_down, down_in, range_up, up_in,
                                       range_in_coupon, range_out_coupon, cash_ccy, trade_direction, fx_fixing, cash_settle)
vol_surface = ConstantVolSurface(today, 0.15)
dev_valuation(inst, today, fx_fwd_crv, d_crv, vol_surface, daycount=ql.Actual365Fixed())


npv: 151190.47745061634
delta: -164.6278431871906
gamma: -67.89848732296377
vega: -8997.126761130712
theta: 749.9869052948488
rho(market): -1807.2924003985825
rho(pillar_rate): -1801.6724412599287
phi(market): 1400.4145993957936
phi(pillar_rate): 1395.9394214567728


## Validation

In [10]:
from nemesis.products.commodity.pm_forward_curve import PMForwardCurve
from nemesis.products.fx.fx_implied_curve import FXImpliedAssetCurve
from nemesis.products.fx.fx_vol_surface import FXVolSurface
from nemesis.utils import *
from nemesis.products.rates import *
from nemesis.market.curves import InterpTypes
from nemesis.market.volatility.const_vol_surface import ConstantVolSurface
from nemesis.utils.calendar import JointCalendar

from nemesis.products.fx.fx_vanilla_option import FXVanillaOption
from nemesis.products.fx.fx_digital_option import FXBinaryOption, FXDigitalOption
from nemesis.products.fx.fx_knock_option import FXKnockOption
from nemesis.products.fx.fx_range_accrual_option import FXRangeAccrualOption

####################################################################
#  NEMESIS ALPHA Version 0.1.0 - This build: 24 Jan 2025 at 10:42 #
####################################################################



In [11]:
def val_valuation(option, value_dt, forward_curve, domestic_curve, vol_surface, notional, spot, strike, dc_type=DayCountTypes.ACT_365F):
    npv = option.value(value_dt, forward_curve, domestic_curve, vol_surface, dc_type)["value"] * notional / strike
    delta = option.delta(value_dt, forward_curve, domestic_curve, vol_surface, dc_type, bump=1) * notional / strike
    gamma = option.gamma(value_dt, forward_curve, domestic_curve, vol_surface, dc_type, bump=1) * notional / strike * spot * 0.01
    vega = option.vega(value_dt, forward_curve, domestic_curve, vol_surface, dc_type, bump=1) * notional / strike
    theta = option.theta(value_dt, forward_curve, domestic_curve, vol_surface, dc_type, bump=1) * notional / strike
    rho1 = option.rho(value_dt, forward_curve, domestic_curve, vol_surface, dc_type, bump=100, bump_type="market") * notional / strike * 100
    rho2 = option.rho(value_dt, forward_curve, domestic_curve, vol_surface, dc_type, bump=100, bump_type="pillar") * notional / strike * 100
    phi1 = option.phi(value_dt, forward_curve, domestic_curve, vol_surface, dc_type, bump=100, bump_type="market") * notional / strike * 100
    phi2 = option.phi(value_dt, forward_curve, domestic_curve, vol_surface, dc_type, bump=100, bump_type="pillar") * notional / strike * 100

    print(f'npv: {npv}')
    print(f'delta: {delta}')
    print(f'gamma: {gamma}')
    print(f'vega: {vega}')
    print(f'theta: {theta}')
    print(f'rho(market): {rho1}')
    print(f'rho(pillar_rate): {rho2}')
    print(f'phi(market): {phi1}')
    print(f'phi(pillar_rate): {phi2}')

In [12]:
value_dt = Date(11,8,2025)
spot_pm_rate = spot
currency_pair = "XAUUSD"

pm_forward_curve = PMForwardCurve(value_dt, spot_pm_rate, pm_fwd_data, "rate", currency_pair, JointCalendar([CalendarTypes.UNITED_KINGDOM, CalendarTypes.UNITED_STATES]), DayCountTypes.ACT_365F, InterpTypes.FLAT_FWD_RATES)
domestic_curve = QLCurve(value_dt, d_crv, dc_type=DayCountTypes.ACT_360, interp_type=InterpTypes.LINEAR_ZERO_RATES)
foreign_implied_curve = FXImpliedAssetCurve(value_dt, domestic_curve, pm_forward_curve, JointCalendar([CalendarTypes.UNITED_KINGDOM, CalendarTypes.UNITED_STATES]), DayCountTypes.ACT_365F, InterpTypes.LINEAR_ZERO_RATES)
pm_vol_surface = FXVolSurface(value_dt, vol_data, spot_pm_rate, currency_pair, pm_forward_curve, foreign_implied_curve, cal_type=JointCalendar([CalendarTypes.UNITED_KINGDOM, CalendarTypes.UNITED_STATES]), dc_type=DayCountTypes.ACT_365F)

In [13]:
expiry_dt = Date(13,11,2025)
strike_pm_rate = 3200
notional = 1e4
prem_currency = "USD"

o = FXVanillaOption(expiry_dt, strike_pm_rate, currency_pair, OptionTypes.EUROPEAN_CALL, notional, prem_currency, JointCalendar([CalendarTypes.UNITED_KINGDOM, CalendarTypes.UNITED_STATES]), spot_days=2)
val_valuation(o, value_dt, pm_forward_curve, domestic_curve, pm_vol_surface, notional, spot_pm_rate, strike_pm_rate, DayCountTypes.ACT_365F)

npv: 664.8243657925817
delta: 2.43517371378843
gamma: 0.11251073228635278
vega: 15.719033583592346
theta: -1.166079181949442
rho(market): 20.119922393357914
rho(pillar_rate): 20.066865942835044
phi(market): -21.91018050772526
phi(pillar_rate): -21.852273578653623


In [14]:
expiry_dt = Date(13,11,2025)
strike_pm_rate = 3200
cash = 3e4
cash_currency = "USD"
pay_on_equal = False
cash_settle = True

o = FXBinaryOption(expiry_dt, strike_pm_rate, currency_pair, OptionTypes.BINARY_CALL, cash, cash_currency, pay_on_equal, JointCalendar([CalendarTypes.UNITED_KINGDOM, CalendarTypes.UNITED_STATES]), spot_days=2, cash_settle=cash_settle)
val_valuation(o, value_dt, pm_forward_curve, domestic_curve, pm_vol_surface, 1, spot_pm_rate, 1, DayCountTypes.ACT_365F)

npv: 22366.585560095296
delta: 37.033463740954176
gamma: -3.7451171374414116
vega: -480.1881314932434
theta: 40.85719475346559
rho(market): 259.57650909244876
rho(pillar_rate): 258.7004465673126
phi(market): -319.78409719644696
phi(pillar_rate): -318.7420805938509


In [15]:
expiry_dt = Date(13,11,2025)
strike_pm_rate = 3200
left_in = False
coupon_left = 3e4
coupon_right = 1e4
cash = 3e4
cash_currency = "USD"
cash_settle = True

o = FXDigitalOption(expiry_dt, strike_pm_rate, currency_pair, left_in, coupon_left, coupon_right, cash_currency, JointCalendar([CalendarTypes.UNITED_KINGDOM, CalendarTypes.UNITED_STATES]), spot_days=2, cash_settle=cash_settle)
val_valuation(o, value_dt, pm_forward_curve, domestic_curve, pm_vol_surface, 1, spot_pm_rate, 1, DayCountTypes.ACT_365F)

npv: 14747.049069556724
delta: -24.68897583639773
gamma: 2.486611865606392
vega: 320.1254209954968
theta: -23.660515101775673
rho(market): -252.90416839532463
rho(pillar_rate): -252.0970435789759
phi(market): 213.18939813096404
phi(pillar_rate): 212.49472039590182


In [16]:
expiry_dt = Date(13,11,2025)
strike_pm_rate = 3200
barrier_pm_rate = 3100
notional = 1e6
notional_currency = "USD"
barrier_type = "upout"
barrier_at_coupon = True
flavor = "put"

o = FXKnockOption(expiry_dt, strike_pm_rate, barrier_pm_rate, barrier_type, barrier_at_coupon, currency_pair, flavor, notional, notional_currency, JointCalendar([CalendarTypes.UNITED_KINGDOM, CalendarTypes.UNITED_STATES]), spot_days=2)
const_vol_surface = ConstantVolSurface(value_dt, JointCalendar([CalendarTypes.UNITED_KINGDOM, CalendarTypes.UNITED_STATES]), DayCountTypes.ACT_365F, 0.15)
val_valuation(o, value_dt, pm_forward_curve, domestic_curve, const_vol_surface, 1, spot_pm_rate, 1, DayCountTypes.ACT_365F)

npv: 9062.909916567049
delta: -64.04819994713762
gamma: 11.7217302613426
vega: 1586.7069421988845
theta: -125.82241891735248
rho(market): -577.6872249344979
rho(pillar_rate): -575.795019117686
phi(market): 553.2449160929227
phi(pillar_rate): 551.4249932358216


In [17]:
start_dt = Date(10,11,2025)
end_dt = Date(13,11,2025)
calendar = Calendar(JointCalendar([CalendarTypes.UNITED_KINGDOM, CalendarTypes.UNITED_STATES]))
schedule = Schedule(start_dt, end_dt, FrequencyTypes.DAILY, JointCalendar([CalendarTypes.UNITED_KINGDOM, CalendarTypes.UNITED_STATES]), BusDayAdjustTypes.MODIFIED_FOLLOWING, DateGenRuleTypes.FORWARD).adjusted_dts
payment_dt = calendar.add_business_days(end_dt, 2)
notional = 1e6
cash_currency = "USD"
cash_settle = True

range_down = 3250
down_in = True
range_up = 3350
up_in = True
range_in_coupon_rate = 1
range_out_coupon_rate = 0
range_in_coupon = range_in_coupon_rate * notional
range_out_coupon = range_out_coupon_rate * notional

fx_fixing = pd.Series([spot], index=[today])

In [18]:
o = FXRangeAccrualOption(schedule, payment_dt, range_down, range_up, down_in, up_in, range_in_coupon, range_out_coupon, currency_pair, cash_currency, fx_fixing, JointCalendar([CalendarTypes.UNITED_KINGDOM, CalendarTypes.UNITED_STATES]), cash_settle)
const_vol_surface = ConstantVolSurface(value_dt, JointCalendar([CalendarTypes.UNITED_KINGDOM, CalendarTypes.UNITED_STATES]), DayCountTypes.ACT_365F, 0.15)
val_valuation(o, value_dt, pm_forward_curve, domestic_curve, const_vol_surface, 1, spot_pm_rate, 1, DayCountTypes.ACT_365F)

npv: 151190.4774506171
delta: -164.6278412954416
gamma: -69.16307233041152
vega: -8997.126761130785
theta: 749.9869052947906
rho(market): -1807.292400398568
rho(pillar_rate): -1801.6724412593464
phi(market): 1400.4145993957645
phi(pillar_rate): 1395.9394214562053
